In [24]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [25]:
gmsh.open("cylinder.geo")

Info    : Reading 'cylinder.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.00940666s, CPU 0.001026s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.338333s, CPU 0.031104s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.0128854s, CPU 0.005142s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0414657s, CPU 0.012413s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.08088 (

In [26]:
mat = FEM.material("body", ν=0.3)
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 1443)


LowLevelFEM.Problem("cylinder", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 8030)

In [27]:
csx(x, y, z) = x
csy(x, y, z) = y
csyy(x, y, z) = -y
cs = FEM.CoordinateSystem([csx, csy, 0], [csyy, csx, 0])
Q = FEM.rotateNodes(problem, "body", cs);

In [28]:
supp = FEM.displacementConstraint("left", uy=0, uz=0)

("left", 0 + 1im, 0, 0)

In [29]:
fx(x, y, z) = -y
fy(x, y, z) = x
load = FEM.load("right", fx=fx, fy=fy, fz=3)

("right", Main.fx, Main.fy, 3)

In [30]:
f2x(x, y, z) = -y * z / 100000 - 0.3 * x / 10000
f2y(x, y, z) = x * z / 100000 - 0.3 * y / 10000
f2z(x, y, z) = 0.01 / 100 * z
torque = FEM.field("body", fx=f2x, fy=f2y, fz=f2z)
q = FEM.vectorField(problem, [torque]);

In [31]:
f2x(x, y, z) = x / 10000
f2y(x, y, z) = y / 10000
f2z(x, y, z) = 0 * 0.01 / 100 * z
torque = FEM.field("body", fx=f2x, fy=f2y, fz=f2z)
q2 = FEM.vectorField(problem, [torque]);

In [32]:
q = Q' * q

24090-element Vector{Float64}:
 -0.0003
  0.01
  0.01
 -0.0003
  1.2037062152420224e-35
  0.0
 -0.00029999999999999905
  0.010000000000000002
  0.010000000000000002
 -0.0002999999999999999
  ⋮
 -0.00023921778586815908
  0.007334409569432159
  0.009197990286735293
 -0.00019965969933294055
  0.00021250489881176056
  0.00031930063932040706
 -0.00017827117557701422
  0.0021195073637302157
  0.003566769597277784

In [33]:
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [34]:
point = FEM.displacementConstraint("A", uz=0)
n = FEM.constrainedDoFs(problem, [point])[1] ÷ 3
n = 43

43

In [35]:
reshape(S[n*9-8:n*9], 3, 3)

3×3 Matrix{Float64}:
  3.77427    11.2383   -0.0241606
 11.2383      8.42722   7.82389
 -0.0241606   7.82389  23.6604

In [36]:
#=
using SparseArrays
struct Transformation
    T::SparseMatrixCSC{Float64}
    non::Int64
    dim::Int64
end

import Base.transpose
function transpose(A::Transformation)
    return Transformation(transpose(A.T), A.non, A.dim)
end

import Base.adjoint
function adjoint(A::Transformation)
    return Transformation(adjoint(A.T), A.non, A.dim)
end

import Base.*
function *(A::Transformation, B)
    n = size(B, 1)
    m = size(B, 2)
    non = A.non
    dim = A.dim
    if dim * non == n
        if m == n
            return dropzeros!(A.T * B)
        else
            return A.T * B
        end
    elseif 9non == n
        C = zeros(3non, 3)
        E = zeros(n, m)
        T = []
        I = []
        J = []
        V = []
        T1 = zeros(9)
        I0 = [1, 2, 3, 1, 2, 3, 1, 2, 3]
        J0 = [1, 1, 1, 2, 2, 2, 3, 3, 3]
        if dim == 2
            for i in 1:non
                T1 = [A.T[2i-1, 2i-1], A.T[2i, 2i-1], 0, A.T[2i-1, 2i], A.T[2i, 2i], 0, 0, 0, 1]
                Idx = I0 .+ (3i - 3)
                Jdx = J0 .+ (3i - 3)
                append!(I, Idx)
                append!(J, Jdx)
                append!(V, T1)
            end
            fn(x, y) = y
            T = sparse(I, J, V, 3non, 3non, fn)
        else
            T = A.T
        end
        for k in 1:m
            for i in 1:non
                for j = 1:3
                    C[3i-2:3i, j] = B[9i-9+3j-2:9i-9+3j, k]
                end
            end
            D = T * C
            for i in 1:non
                for j = 1:3
                    E[9i-9+3j-2:9i-9+3j, k] = D[3i-2:3i, j]
                end
            end
        end
        return E
    else
        error("*(A::Transformation, B): size missmatch dim * non = $dim * $non ≠ $n.")
    end
end

function *(B, A::Transformation)
    n = size(B, 1)
    m = size(B, 2)
    non = A.non
    dim = A.dim
    if dim * non == n
        if m == n
            return dropzeros!(A.T * B)
        else
            return A.T * B
        end
    elseif 9non == n
        C = zeros(3, 3non)
        E = zeros(n, m)
        T = []
        I = []
        J = []
        V = []
        T1 = zeros(9)
        I0 = [1, 2, 3, 1, 2, 3, 1, 2, 3]
        J0 = [1, 1, 1, 2, 2, 2, 3, 3, 3]
        if dim == 2
            for i in 1:non
                T1 = [A.T[2i-1, 2i-1], A.T[2i, 2i-1], 0, A.T[2i-1, 2i], A.T[2i, 2i], 0, 0, 0, 1]
                Idx = I0 .+ (3i - 3)
                Jdx = J0 .+ (3i - 3)
                append!(I, Idx)
                append!(J, Jdx)
                append!(V, T1)
            end
            fn(x, y) = y
            T = sparse(I, J, V, 3non, 3non, fn)
        else
            T = A.T
        end
        for k in 1:m
            for i in 1:non
                for j = 1:3
                    C[1:3, 3i-3+j] = B[9i-9+3j-2:9i-9+3j, k]
                end
            end
            D = C * T
            for i in 1:non
                for j = 1:3
                    E[9i-9+3j-2:9i-9+3j, k] = D[1:3, 3i-3+j]
                end
            end
        end
        return E
    else
        error("*(B, A::Transformation): size missmatch dim * non = $dim * $non ≠ $n.")
    end
end
=#

In [37]:
q1 = Q' * q
S2 = FEM.solveStress(problem, q1)
S2 = FEM.elementsToNodes(problem, S2);

In [38]:
S3 = Q' * S * Q;

DimensionMismatch: DimensionMismatch: first dimension of A, 24090, does not match the first dimension of B, 72270

In [39]:
reshape(S2[n*9-8:n*9], 3, 3)

3×3 Matrix{Float64}:
  6.95256   23.0697   -0.047888
 23.0697    16.158     7.71215
 -0.047888   7.71215  26.9332

In [40]:
reshape(S3[n*9-8:n*9], 3, 3)

3×3 Matrix{Float64}:
 -0.0384409    0.00346762   0.047888
  0.00346762  -0.0250046    7.71215
  0.047888     7.71215     19.981

In [41]:
u = FEM.showDoFResults(problem, q1, :uvec)
ux = FEM.showDoFResults(problem, q1, :ux)
uy = FEM.showDoFResults(problem, q1, :uy)
uz = FEM.showDoFResults(problem, q1, :uz)
sx = FEM.showDoFResults(problem, S2, :sx)
sy = FEM.showDoFResults(problem, S2, :sy)
sz = FEM.showDoFResults(problem, S2, :sz)
sxy = FEM.showDoFResults(problem, S2, :sxy)
syz = FEM.showDoFResults(problem, S2, :syz, visible=false)
szx = FEM.showDoFResults(problem, S2, :szx)

9

In [42]:
s2x = FEM.showDoFResults(problem, S3, :sx)
s2y = FEM.showDoFResults(problem, S3, :sy)
s2z = FEM.showDoFResults(problem, S3, :sz)
s2xy = FEM.showDoFResults(problem, S3, :sxy)
s2yz = FEM.showDoFResults(problem, S3, :syz, visible=true)
s2zx = FEM.showDoFResults(problem, S3, :szx)

15

In [43]:
p1 = FEM.plotOnPath(problem, "path1", s2yz)
p2 = FEM.plotOnPath(problem, "path2", s2yz)

17

In [44]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [45]:
gmsh.finalize()